In [1]:
import jax
import time
import pypomp
import unittest
import jax.numpy as np
import pandas as pd

import pykalman
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from pypomp.mop import mop
from pypomp.pfilter import pfilter
from pypomp.internal_functions import _mop_internal
from pypomp.internal_functions import _pfilter_internal
from pypomp.internal_functions import _pfilter_internal_mean


/var/folders/5v/yphhcgv17dz3dwr2sc43_pwm0000gn/T/ipykernel_7292/2920153759.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Test the Linear Gaussian Model: A comparison with Kalman-filtering algorithm

Model Setup

Kalman filter deals with the dynamic system:
\begin{align}
    x_t &= A x_{t-1} + w_t \\
    \text{where} \quad
    x_t &\text{ is the current state vector.} \nonumber \\
    A &\text{ is the state transition matrix.} \nonumber \\
    w_t &\sim \mathcal{N}(0, Q) \text{ is the process noise, normally distributed with mean 0 and covariance } Q. \nonumber \\
    y_t &= C x_t + v_t \\
    \text{where} \quad
    y_t &\text{ is the current observation vector.} \nonumber \\
    C &\text{ is the observation matrix.} \nonumber \\
    v_t &\sim \mathcal{N}(0, R) \text{ is the observation noise, normally distributed with mean 0 and covariance} R. \nonumber
\end{align}


(1) Set T (Time Length) to be 1000 and generate linear Gaussian states and observations:

In [5]:
def get_thetas(theta):
    A = theta[0:4].reshape(2, 2)
    C = theta[4:8].reshape(2, 2)
    Q = theta[8:12].reshape(2, 2)
    R = theta[12:16].reshape(2, 2)
    return A, C, Q, R

def transform_thetas(A, C, Q, R):
    return np.concatenate([A.flatten(), C.flatten(), Q.flatten(), R.flatten()])


fixed = False
key = jax.random.PRNGKey(111)
angle = 0.2
angle2 = angle if fixed else -0.5
A = np.array([[np.cos(angle2), -np.sin(angle)],
             [np.sin(angle), np.cos(angle2)]])
C = np.eye(2)
Q = np.array([[1, 1e-4],
             [1e-4, 1]]) #/ 100
R = np.array([[1, .1],
            [.1, 1]]) #/ 10
     
theta = transform_thetas(A, C, Q, R)

def generate_data(N, key):
    xs = []
    ys = []
    x = np.ones(2)
    for i in tqdm(range(N)):
        key, subkey = jax.random.split(key)
        x = jax.random.multivariate_normal(key=subkey, mean=A @ x, cov=Q)
        key, subkey = jax.random.split(key)
        y = jax.random.multivariate_normal(key=subkey, mean=C @ x, cov=R)
        xs.append(x)
        ys.append(y)
    xs = np.array(xs)
    ys = np.array(ys)
    return xs, ys, key

def custom_rinit(theta, J, covars=None):
    return np.ones((J, 2))

def custom_rproc(state, theta, key, covars=None):
    A, C, Q, R = get_thetas(theta)
    key, subkey = jax.random.split(key)
    return jax.random.multivariate_normal(key=subkey,
                                          mean=A @ state, cov=Q)
def custom_dmeas(y, preds, theta):
    A, C, Q, R = get_thetas(theta)
    return jax.scipy.stats.multivariate_normal.logpdf(y, preds, R)

rinit = custom_rinit
rproc = custom_rproc
dmeas = custom_dmeas
rprocess = jax.vmap(custom_rproc, (0, None, 0, None))
dmeasure = jax.vmap(custom_dmeas, (None, 0, None))
rprocesses = jax.vmap(custom_rproc, (0, 0, 0, None))
dmeasures = jax.vmap(custom_dmeas, (None, 0, 0))

In [3]:
def logmeanexp(x):
   x_array = np.array(x)
   x_max = np.max(x_array)
   log_mean_exp = np.log(np.mean(np.exp(x_array - x_max))) + x_max
   return log_mean_exp

Set J=10,000 (number of particles) and compare the estimated log-likelihood between Kalman filtering and the log-mean-exponential computed over 100 replications for various methods, including classical particle filtering and MOP, etc.

In [8]:
xs, ys, key = generate_data(1000, key)
kf = pykalman.KalmanFilter(transition_matrices=A, observation_matrices=C, 
                        transition_covariance=Q, observation_covariance=R)
print("kf loglik =", kf.loglikelihood(ys))

100%|██████████| 1000/1000 [00:00<00:00, 2654.98it/s]


kf loglik = -3759.064178842396


In [9]:
loglike = []
for i in range(100):  
    key, subkey = jax.random.split(key)
    pfilter_val = -_pfilter_internal(theta, ys, J = 10000, rinit = rinit, rprocess = rprocess, dmeasure = dmeasure, covars = None,
                                   key= key, thresh = -1)
    loglike.append(pfilter_val)

loglike_ = np.array(loglike)
print("Logmeanexp of Particle Filtering =", logmeanexp(loglike))
print("difference between Kalman-Filtering and logmeanexp of Particle Filtering =", kf.loglikelihood(ys) - (logmeanexp(loglike)))

Logmeanexp of Particle Filtering = -3759.153
difference between Kalman-Filtering and logmeanexp of Particle Filtering = 0.08886719


By calculating the difference between Kalman-filtering and Paticle Filtering algorithm, we discovered that the value of difference can reach 0.089, indicating that we get a reasonable inference result from the MOP algorithm.

In [10]:
alphas = [0, 0.1, 0.3, 0.6, 0.9, 1]  
results = []

for alpha in alphas:
    loglike_mop = []
    for i in range(100):  
        key, subkey = jax.random.split(key)
        mop_val = -_mop_internal(theta, ys, J=10000, rinit=rinit, rprocess=rprocess, dmeasure=dmeasure, covars=None,
                                 key=key, alpha=alpha)
        loglike_mop.append(mop_val)

    loglike_mop = np.array(loglike_mop)
    logmeanexp_val = logmeanexp(loglike_mop)
    difference = kf.loglikelihood(ys) - logmeanexp_val
    
    results.append((alpha, logmeanexp_val, difference))
    print(f"Alpha: {alpha}, Logmeanexp: {logmeanexp_val}, Difference: {difference}")


Alpha: 0, Logmeanexp: -3759.23095703125, Difference: 0.166748046875
Alpha: 0.1, Logmeanexp: -3759.15087890625, Difference: 0.086669921875
Alpha: 0.3, Logmeanexp: -3759.229248046875, Difference: 0.1650390625
Alpha: 0.6, Logmeanexp: -3759.072265625, Difference: 0.008056640625
Alpha: 0.9, Logmeanexp: -3759.102294921875, Difference: 0.0380859375
Alpha: 1, Logmeanexp: -3759.331298828125, Difference: 0.26708984375


By calculating the difference between the Kalman-filtering and MOP algorithm estimates for different values of $\alpha$, we observed that the maginitude varies without an obvious trend as $\alpha$ increases from 0 to 1. The maginitude reaches its minimum with 0.008 when $\alpha = 0.6$, suggesting that the MOP algorithm has the potential to provide an inference results that are highly consistent with those from the Kalman filtering method when choosing $\alpha$ appropriately.



### Test the Linear Gaussian Model: How estimate logllikehood difference and running time varys among different T and J

In [ ]:
N_values = [1000, 5000]  
J_values = [10, 100, 1000, 10000]  

results = []
key = jax.random.PRNGKey(112)

for N in N_values:
    for J in J_values:
        print(f"Running with N={N}, J={J}...")
        
        xs, ys, key = generate_data(N, key)
        pf_loglik_arr = []
        mop_loglik_arr = []
        elapsed_time1_arr = []
        elapsed_time2_arr = []
        
        for i in range(100):  
            start_time = time.time()
            pf_val = -pfilter(J = J, rinit = rinit, rprocess = rprocess, dmeasure = dmeasure, theta = theta, ys = ys, thresh = 0, key = key)
            pf_loglik_arr.append(pf_val)
            elapsed_time1_arr.append(time.time() - start_time)

            start_time2 = time.time()
            mop_val = -mop(J = J, rinit = rinit, rprocess = rprocess, dmeasure = dmeasure, theta = theta, ys = ys, alpha = 0.9, key = key)
            mop_loglik_arr.append(mop_val)
            elapsed_time2_arr.append(time.time() - start_time2)
        
        pf_loglik_arr = np.array(pf_loglik_arr)
        mop_loglik_arr = np.array(mop_loglik_arr)
        elapsed_time1_arr = np.array(elapsed_time1_arr)
        elapsed_time2_arr = np.array(elapsed_time2_arr)

        results.append({
            'N': N, 
            'J': J, 
            'pf_loglik': logmeanexp(pf_loglik_arr), 
            'time_pfilter': np.mean(elapsed_time1_arr), 
            'mop_loglik': logmeanexp(mop_loglik_arr), 
            'time_mop': np.mean(elapsed_time2_arr), 
        })

In [24]:
results

[{'N': 1000,
  'J': 10,
  'pf_loglik': Array(-4059.384, dtype=float32),
  'time_pfilter': Array(0.00262911, dtype=float32),
  'mop_loglik': Array(-4044.6467, dtype=float32),
  'time_mop': Array(0.01157162, dtype=float32)},
 {'N': 1000,
  'J': 100,
  'pf_loglik': Array(-3766.4568, dtype=float32),
  'time_pfilter': Array(0.03207133, dtype=float32),
  'mop_loglik': Array(-3776.5305, dtype=float32),
  'time_mop': Array(0.0302381, dtype=float32)},
 {'N': 1000,
  'J': 1000,
  'pf_loglik': Array(-3747.1902, dtype=float32),
  'time_pfilter': Array(0.3003823, dtype=float32),
  'mop_loglik': Array(-3744.5942, dtype=float32),
  'time_mop': Array(0.4241002, dtype=float32)},
 {'N': 1000,
  'J': 10000,
  'pf_loglik': Array(-3724.4568, dtype=float32),
  'time_pfilter': Array(2.2342062, dtype=float32),
  'mop_loglik': Array(-3724.7112, dtype=float32),
  'time_mop': Array(3.1925905, dtype=float32)},
 {'N': 1000,
  'J': 100000,
  'pf_loglik': Array(-3797.8877, dtype=float32),
  'time_pfilter': Array(17.

Result (To Be Continued):

\begin{array}{|c|c|c|c|c|c|}
\hline
\textbf{N} & \textbf{J} & \textbf{Particle Filter Loglik} & \textbf{Time (Particle Filter)} & \textbf{MOP Loglik} & \textbf{Time (MOP)} \\
\hline
1000 & 10 & -4059.384 & 0.00262911 & -4044.6467 & 0.01157162 \\
1000 & 100 & -3766.4568 & 0.03207133 & -3776.5305 & 0.0302381 \\
1000 & 1000 & -3747.1902 & 0.3003823 & -3744.5942 & 0.4241002 \\
1000 & 10000 & -3724.4568 & 2.2342062 & -3724.7112 & 3.1925905 \\
1000 & 100000 & -3797.8877 & 17.238829 & -3797.6775 & 16.385502 \\
5000 & 10 & -20326.46 & 0.02121412 & -20303.664 & 0.02167264 \\
5000 & 100 & -18856.922 & 0.09896883 & -18853.13 & 0.10377918 \\
5000 & 1000 & -18803.223 & 1.323793 & -18808.762 & 1.9390032 \\
5000 & 10000 & -18672.562 & 10.692378 & -18672.344 & 15.778171 \\
\hline
\end{array}
